In [1]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Attention

vocab_size_input = 10000 + 1
vocab_size_output = 6000 + 1
embedding_dim = 128
units=256

2025-07-29 21:17:13.514996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753834634.211499    7149 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753834634.377487    7149 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753834635.947466    7149 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753834635.947519    7149 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753834635.947523    7149 computation_placer.cc:177] computation placer alr

In [2]:
# ENCODER LAYERS
encoder_inputs = Input(shape=(100,))
enc_emb = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim)(encoder_inputs)

encoder_lstm = LSTM(units, return_sequences=True, return_state=True)
encoder_output, state_h, state_c = encoder_lstm(enc_emb)

# DECODER LAYERS
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(vocab_size_output, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# ATTENTION LAYERS
attention = Attention()
context_vector = attention([decoder_outputs, encoder_output])

decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, context_vector])
decoder_dense = TimeDistributed(Dense(vocab_size_output, activation='softmax'))
final_output = decoder_dense(decoder_concat_input)

model = Model([encoder_inputs, decoder_inputs], final_output)

I0000 00:00:1753834662.442389    7149 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1794 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [30]:
def F1Score(y_true, y_pred):
    # y_true: (batch_size, seq_len)
    # y_pred: (batch_size, seq_len, vocab_size)
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.float32)
    vocab_size = tf.shape(y_pred)[-1]

    # One-hot encode y_true to match y_pred's shape
    y_true_oh = tf.one_hot(y_true, depth=vocab_size)  # (batch_size, seq_len, vocab_size)

    # Convert y_pred to binary predictions (argmax)
    y_pred_bin = tf.one_hot(tf.argmax(y_pred, axis=-1), depth=vocab_size)

    tp = tf.reduce_sum(y_true_oh * y_pred_bin)
    fp = tf.reduce_sum((1 - y_true_oh) * y_pred_bin)
    fn = tf.reduce_sum(y_true_oh * (1 - y_pred_bin))

    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())

    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
    return f1

In [6]:
model.compile(
	optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3),
	loss=tf.keras.losses.sparse_categorical_crossentropy,
	metrics=['accuracy', 'precision', 'recall', F1Score, 'AUC']
)

In [22]:
# Parâmetros que você usou na outra sessão
BATCH = 16384         # ou outro valor que você tenha definido
MAXLEN_DOC = 100      # comprimento da sequência do document
MAXLEN_SUM = 30       # comprimento da sequência do summary

# Reconstrua o element_spec manualmente:
element_spec = {
    'document': tf.TensorSpec(shape=(BATCH, MAXLEN_DOC), dtype=tf.int64),
    'summary' : tf.TensorSpec(shape=(BATCH, MAXLEN_SUM),  dtype=tf.int64),
}


In [23]:
ds = tf.data.Dataset.load(
    "vectorized_gigaword_ds",
    element_spec=element_spec
)

In [24]:
print(ds)

<_LoadDataset element_spec={'document': TensorSpec(shape=(16384, 100), dtype=tf.int64, name=None), 'summary': TensorSpec(shape=(16384, 30), dtype=tf.int64, name=None)}>


In [28]:
VALIDATION_SPLIT = 0.2
SHUFFLE_BUFFER   = 10000
BATCH_SIZE       = 64

def make_training_example(x):
    enc_in = x['document']                         # (MAXLEN_DOC,)
    dec_in = x['summary'][:-1]                     # (MAXLEN_SUM-1,)
    dec_tr = x['summary'][1:]                      # (MAXLEN_SUM-1,)
    return ( (enc_in, dec_in), dec_tr )

paired = ds.map(make_training_example, num_parallel_calls=tf.data.AUTOTUNE)
shuffled = paired.shuffle(SHUFFLE_BUFFER, reshuffle_each_iteration=False)
total = tf.data.experimental.cardinality(shuffled).numpy()
val_count = int(total * VALIDATION_SPLIT)

val_ds = (
    shuffled
    .take(val_count)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

train_ds = (
    shuffled
    .skip(val_count)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [25]:
for batch in ds.take(1):
	print(batch['document'].shape)

(16384, 100)


2025-07-29 19:04:45.027012: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# Debug: print a batch from train_ds to check shapes
for (inputs, targets) in train_ds.take(1):
    print("Encoder input shape:", inputs[0].shape)
    print("Decoder input shape:", inputs[1].shape)
    print("Target shape:", targets.shape)

model.fit(
    train_ds,
    epochs=30,
    validation_data=val_ds
)

Encoder input shape: (64, 100)
Decoder input shape: (64, 29)
Target shape: (64, 29)
Epoch 1/30


ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.

In [36]:
for (inputs, targets) in train_ds.take(1):
    print("Encoder input shape:", inputs[0].shape)
    print("Decoder input shape:", inputs[1].shape)
    print("Target shape:", targets.shape)

Encoder input shape: (64, 100)
Decoder input shape: (64, 29)
Target shape: (64, 29)
